# **Import Libraries**

In [ ]:
#Set path to MAIN FOLDER OF EXPERIMENT
#cd /path/to/EXPERIMENT_FOLDER/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/PSR

/content/drive/MyDrive/PSR


In [3]:
#Import Models APIs
from tensorflow.keras.layers import Input, Average, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import inception_resnet_v2
from tensorflow.keras.applications import efficientnet 
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
import itertools
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os

print("LIBRARIES LOADED")

LIBRARIES LOADED


In [11]:
#Tweakable parameters
MODEL_KIND = "teacher_model"
MODEL_NAME = "EnsembleModel"
MODEL_PATH = 'models/' + MODEL_KIND + '/' + MODEL_NAME

BATCH_SIZE = 4
IMAGE_SIZE = (224, 224)

ENSEMBLE_FILES = ['EfficientNetB7',
                  'DenseNet121',
                  'DenseNet201',
                  'InceptionResNetV2', 
                  'InceptionV3']

#Data paths
MAIN_DATA_DIR = "ds/"
TRAIN_DATA_DIR = MAIN_DATA_DIR + "train/"
TEST_DATA_DIR = MAIN_DATA_DIR + "test/"
VALIDATION_DATA_DIR = MAIN_DATA_DIR + "val/"

#Model path
TEACHER_MODEL_PATH = 'models/teacher_model/'

print("ALL REQUERED PATHS SET")

ALL REQUERED PATHS SET


In [13]:
#Save Model Function
def save_m(model, directory, model_name):
    if not os.path.exists(directory):
        os.makedirs(directory)
    model.save(directory + "/" + model_name + ".h5")

#Load model Function
def load_m(directory, model_name):
    if not os.path.exists(directory):
        print("Model File Does Not Exist!!")
        return 
    model = load_model(directory + "/" + model_name + ".h5", custom_objects=processing_layers)
    return model

#Get Ensemble Function with given input shape
def get_ensemble(all_models, to_ensemble, input_shape=IMAGE_SIZE):
	emodels = []
	ensemble_input = Input(shape=(input_shape[0], input_shape[1], 3), name='ensemble_input')
	for model_name in to_ensemble:
		emodel = all_models[model_name]
		emodel = add_preprocess_layer(emodel, processing_layers[model_name], ensemble_input)
		emodel = rename_all_layers(emodel, model_name)
		emodels.append(emodel)

	outputs = [m.output for m in emodels]
	y = Average()(outputs) 
	model = Model(inputs=ensemble_input , outputs=y, name=MODEL_KIND + '-' + MODEL_NAME)
	return model

#Add preprocessing layer to model
def add_preprocess_layer(emodel, pre_layer, ensemble_input):
	x = pre_layer()(ensemble_input)
	outputs = emodel(x)
	model = Model(inputs = ensemble_input, outputs = outputs)
	return model

#Rename all layers except input layer of model
def rename_all_layers(model, file):
	for i in range(1, len(model.layers)):
		model.layers[i]._name = model.layers[i]._name + '_' + file

	return model

print("ALL CUSTOM FUNCTIONS DEFIEND")

ALL CUSTOM FUNCTIONS DEFIEND


**Preprocessing Layers**

In [7]:
class ENB7_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(ENB7_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = efficientnet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(ENB7_PreprocessLayer, self).get_config()
        return config

class Dense_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(Dense_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = densenet.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(Dense_PreprocessLayer, self).get_config()
        return config

class IRNV2_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(IRNV2_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = inception_resnet_v2.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(IRNV2_PreprocessLayer, self).get_config()
        return config

class INV3_PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self, name="preprocess", **kwargs):
        super(INV3_PreprocessLayer, self).__init__(name=name, **kwargs)
        self.preprocess = inception_v3.preprocess_input

    def call(self, input):
        return self.preprocess(input)

    def get_config(self):
        config = super(INV3_PreprocessLayer, self).get_config()
        return config

processing_layers = {'EfficientNetB7':ENB7_PreprocessLayer, 
                  'DenseNet121':Dense_PreprocessLayer,
                  'DenseNet201':Dense_PreprocessLayer,
                  'InceptionResNetV2':IRNV2_PreprocessLayer,
                  'InceptionV3':INV3_PreprocessLayer}

print("ALL REQUERED PREPROCESSING LAYERS DEFINED")

ALL REQUERED PREPROCESSING LAYERS DEFINED


In [ ]:
teacher_models = {}

for file in ENSEMBLE_FILES:
	teacher_models[file] = {}

#Actual models
teacher_models['DenseNet201'] = load_m(TEACHER_MODEL_PATH + 'DenseNet201', 'DenseNet201')
teacher_models['DenseNet121'] = load_m(TEACHER_MODEL_PATH + 'DenseNet121', 'DenseNet121')
teacher_models['InceptionResNetV2'] = load_m(TEACHER_MODEL_PATH + 'InceptionResNetV2', 'InceptionResNetV2')
teacher_models['InceptionV3'] = load_m(TEACHER_MODEL_PATH + 'InceptionV3', 'InceptionV3')
teacher_models['EfficientNetB7'] = load_m(TEACHER_MODEL_PATH + 'EfficientNetB7', 'EfficientNetB7')
print("[INFO] Load models")

Model File Does Not Exist!!
Model File Does Not Exist!!
Model File Does Not Exist!!
Model File Does Not Exist!!


In [ ]:
#LOAD TEST & VAL DATA
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

if not os.path.exists(VALIDATION_DATA_DIR):
    print("VALIDATION DATA DOES NOT EXITS!")
else:
    print("LOAD VALIDATION SAMPLES...")
    validation_generator = val_datagen.flow_from_directory(
        VALIDATION_DATA_DIR,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=42,
        shuffle=False)

    #CHECK  THE NUMBER OF SAMPLES
    nb_validation_samples = len(validation_generator.filenames)
    if nb_validation_samples == 0:
        print("NO DATA VALIDATION FOUND IN VALIDATION FOLDER!")

print()
if not os.path.exists(TEST_DATA_DIR):
    print("TEST DATA DOES NOT EXITS!")
else:
    print("LOAD TEST SAMPLES...")
    test_generator = test_datagen.flow_from_directory(
                TEST_DATA_DIR,
                target_size=IMAGE_SIZE,
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                seed=42,
                shuffle=False)

    #CHECK  THE NUMBER OF SAMPLES
    nb_test_samples = len(test_generator.filenames)
    if nb_test_samples == 0:
        print("NO DATA TEST FOUND IN TEST FOLDER!")

# **ENSEMBLE MODEL ABLATIONS**

In [ ]:
TOTAL_CLASSES = 199
LEARNING_RATE = 0.001
OPTIMIZER = Adam
BATCH_SIZE = 4
EPOCHS = 30
DROPOUT_RATE = 0.5

print("HYPERPARAMETERS")
print("---------------------")
print("BATCH_SIZE -->", BATCH_SIZE)
print("EPOCHS SET -->", EPOCHS)
print("DROPOUT_RATE -->", DROPOUT_RATE)
print("LEARNING_RATE -->", LEARNING_RATE)
print("OPTIMIZER -->", OPTIMIZER.__name__)

In [ ]:
import itertools

n = 2
max_val_acc = 0
best_ablation = None

while(n <= len(ENSEMBLE_FILES)):
    for to_ensemble in list(itertools.combinations(ENSEMBLE_FILES, n)):
        print("Current Ablation:", to_ensemble)
        model = get_ensemble(teacher_models, to_ensemble)
        model.compile(
            optimizer=Adam(learning_rate=LEARNING_RATE),
            loss = CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        acc = model.evaluate(validation_generator)[1]
        if max_val_acc < acc:
            max_val_acc = acc
            best_ablation = to_ensemble
        
    n += 1

print()
print("Best Ablation:", best_ablation)
print("Highest Accuracy:", max_val_acc)

In [ ]:
#EVALUATION OF BEST ABLATION
teacher_ensemble = get_ensemble(teacher_models, best_ablation)
teacher_ensemble.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss = CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

teacher_ensemble.summary()

In [15]:
model = load_m(MODEL_PATH, MODEL_NAME)

ValueError: ignored

In [ ]:
print("Evaluation on Validation Set")
acc = teacher_ensemble.evaluate(validation_generator)

print("Evaluation on Test Set")
acc = teacher_ensemble.evaluate(test_generator)

In [ ]:
#save model
save_m(teacher_ensemble, TEACHER_MODEL_PATH + MODEL_NAME, MODEL_NAME)
print("[INFO] MODEL AND HISTORY SAVED to ", TEACHER_MODEL_PATH + 'TeacherEnsemble/')